#  The Moving Average Convergence Divergence Strategy (Long-short)

It is a simple and effective trend-following indicator.

The strategy uses:

MACD = 26 day EMA of 'Close' - 12 day EMA of 'Close', and
signal = 9 day EMA of MACD
The trading signals are generated using MACD and signal.

When MACD crosses above signal, we go long on the underlying security
When MACD crosses below signal, then we go short on it

Import the required libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import warnings 

warnings.filterwarnings('ignore')

Fix the backtesting period

In [2]:
# Backtesting period should be of 5 years
end = pd.datetime.now().date()
start = end - pd.Timedelta(days=5*252)

Create a class and create instance attributes/methods which are required for backtesting

In [3]:
# Create a class
class backtesting_MACD:
    
    # Create Instance attribute and methods
    def __init__(self,ticker,start,end,macd_low,macd_high,macd_signal):
        
        self.ticker = ticker
        self.start = start
        self.end = end
        self.macd_low = macd_low
        self.macd_high = macd_high
        self.macd_signal = macd_signal
        
        self.fetch_data()
        self.indicators()
        self.signals()
        self.positions()
        self.returns()
        
    #To fetch the required data from yahoo finance    
    def fetch_data(self):
        self.df = yf.download(self.ticker,self.start,self.end)
    
    # To compute the required indicators Long SMA and Short SMA
    def indicators(self):
        self.df['macd_low_ema'] = self.df['Adj Close'].ewm(span=self.macd_low,adjust=False).mean()
        self.df['macd_high_ema'] = self.df['Adj Close'].ewm(span=self.macd_high,adjust=False).mean()
        self.df['macd_signal_ema'] = self.df['Adj Close'].ewm(span=self.macd_signal,adjust=False).mean()
        self.df['MACD'] = self.df['macd_high_ema'] - self.df['macd_low_ema']
        
    # To generate the long and short entry signal
    # signal 1 for long position and -1 for short position
    def signals(self):
        self.df['signals'] = np.where((self.df['MACD'] > self.df['macd_signal_ema']),1,-1)
        
    # To mark the positions taken
    def positions(self):
        self.df['positions'] = self.df['signals'].shift()
    
    #To compute the returns generated
    def returns(self):
        self.df['BnH Returns'] = self.df['Adj Close'].pct_change()
        self.df['Strategy Returns'] = self.df['positions'] * self.df['BnH Returns']
        # BnH returns is the returns generated if bought and hold it for the mentioned period simply without any strategy
        self.df['BnH Returns'] = 1 + self.df['BnH Returns']
        # Strategy Returns is the returns generated due to implementation of this strategy on this historical data
        self.df['Strategy Returns'] = 1 + self.df['Strategy Returns']
        Strat_Returns = self.df['Strategy Returns'].cumprod()[-1]
        BnH_Returns = self.df['BnH Returns'].cumprod()[-1]
        print('Buy and Hold Returns :', self.df['BnH Returns'].cumprod()[-1])
        print('Strategy Returns :', self.df['Strategy Returns'].cumprod()[-1])
        return Strat_Returns, BnH_Returns
    

In [4]:
# Try with SPY index by passing relative range as 25 and internal bar strength as 0.3, this will return the buy and hold return & strategy return 
Nifty_backtesting_MACD = backtesting_MACD('^NSEI',start,end,26,12,9)

[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 1.4804639313307522
Strategy Returns : 0.5807465000673344


Optimise the strategy by considering few other stocks

In [5]:
Returns=[]
stocks=[]

stocks_list = ['BHARTIARTL.NS','NTPC.NS','MARUTI.NS','HINDALCO.NS','ONGC.NS','KOTAKBANK.NS','BAJAJ-AUTO.NS','COALINDIA.NS',
                      'ITC.NS','ULTRACEMCO.NS','CIPLA.NS','LT.NS','INDUSINDBK.NS','HDFCLIFE.NS','TATACONSUM.NS','RELIANCE.NS',
                      'TATASTEEL.NS','ICICIBANK.NS','GRASIM.NS','NESTLEIND.NS','TCS.NS','TECHM.NS','SHREECEM.NS','BAJFINANCE.NS',
                      'BAJAJFINSV.NS','WIPRO.NS','BRITANNIA.NS','HEROMOTOCO.NS','TITAN.NS']

for k in stocks_list:
    a = backtesting_MACD(k,start,end,26,12,9)
    stocks.append(k)
    Returns.append(a.returns())    
            
Strat = [x[0] for x in Returns]
BnH = [x[1] for x in Returns]
    

[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 2.128572863718729
Strategy Returns : 0.30000945761077785
Buy and Hold Returns : 2.128572863718729
Strategy Returns : 0.30000945761077785
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 1.309909229207697
Strategy Returns : 0.566644771652086
Buy and Hold Returns : 1.309909229207697
Strategy Returns : 0.566644771652086
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.9702059796571013
Strategy Returns : 0.6722856954227002
Buy and Hold Returns : 0.9702059796571013
Strategy Returns : 0.6722856954227002
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 2.2160875059875003
Strategy Returns : 0.23499774410188365
Buy and Hold Returns : 2.2160875059875003
Strategy Returns : 0.23499774410188365
[*********************100%***********************]  1 of 1 completed
Buy and Hold Retu

In [6]:
# Create a dataframe and validate the returns generated for various indices with different EMA range
Results = pd.DataFrame({'Stocks':stocks,'Strategy Returns':Strat,'BnH Returns':BnH})


In [7]:
# Shows the record in descending order which has maximum return 
Results.sort_values(by='Strategy Returns',inplace=True,ascending=False)

Note : This is simple strategy to learn how to backtest and optimize the parameters by reusing the code. This can be enhanced by coding the performance metrics such as max drawdown, sharpe ratio, hit ratio e.t.c.

In [8]:
Results.head(20)

,Stocks,Strategy Returns,BnH Returns
8,ITC.NS,0.963377,0.783073
7,COALINDIA.NS,0.894369,0.781641
27,HEROMOTOCO.NS,0.770322,0.881531
26,BRITANNIA.NS,0.708164,1.098997
12,INDUSINDBK.NS,0.680340,0.458477
2,MARUTI.NS,0.672286,0.970206
1,NTPC.NS,0.566645,1.309909
6,BAJAJ-AUTO.NS,0.536638,1.413073
22,SHREECEM.NS,0.525512,1.375171
13,HDFCLIFE.NS,0.517167,1.351635
